<a href="https://colab.research.google.com/github/DinurakshanRavichandran/Visio-Glance/blob/Fundus-eye-disease-detection/custom_one.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, classification_report

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Constants
IMG_SIZE = 224
CATEGORIES = ['cataract', 'diabetic_retinopathy', 'glaucoma', 'normal']
BATCH_SIZE = 32
EPOCHS = 50  # Increased to allow for early stopping
DATA_DIR = '/content/drive/MyDrive/data set/dataset'

# ================== Enhanced Data Augmentation ==================
train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.3,
    zoom_range=0.3,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True,
    vertical_flip=True,
    channel_shift_range=50,
    fill_mode='constant',
    validation_split=0.2
)

# Train generator with explicit class order
train_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    classes=CATEGORIES  # Critical fix for label alignment
)

# Validation generator
val_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    classes=CATEGORIES  # Critical fix for label alignment
)

# ================== Class Weight Calculation ==================
y_train = train_generator.classes
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weight_dict = {i: class_weights[i] for i in range(len(CATEGORIES))}

# ================== Dual Input Generator ==================
def dual_input_generator(generator):
    for x, y in generator:
        yield ((x, x), y)  # Dual input format

train_dual_dataset = tf.data.Dataset.from_generator(
    lambda: dual_input_generator(train_generator),
    output_signature=(
        (tf.TensorSpec(shape=(None, IMG_SIZE, IMG_SIZE, 3), dtype=tf.float32),
         tf.TensorSpec(shape=(None, IMG_SIZE, IMG_SIZE, 3), dtype=tf.float32)),
        tf.TensorSpec(shape=(None, len(CATEGORIES)), dtype=tf.float32)
    )
).prefetch(tf.data.AUTOTUNE)

val_dual_dataset = tf.data.Dataset.from_generator(
    lambda: dual_input_generator(val_generator),
    output_signature=(
        (tf.TensorSpec(shape=(None, IMG_SIZE, IMG_SIZE, 3), dtype=tf.float32),
         tf.TensorSpec(shape=(None, IMG_SIZE, IMG_SIZE, 3), dtype=tf.float32)),
        tf.TensorSpec(shape=(None, len(CATEGORIES)), dtype=tf.float32)
    )
).prefetch(tf.data.AUTOTUNE)

# ================== Enhanced Model Architecture ==================
def create_model():
    # Base models
    vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
    resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

    # Freeze initial layers
    for layer in vgg_base.layers:
        layer.trainable = False
    for layer in resnet_base.layers:
        layer.trainable = False

    # Optional: Unfreeze last few layers for fine-tuning
    # for layer in vgg_base.layers[-8:]:
    #     layer.trainable = True
    # for layer in resnet_base.layers[-8:]:
    #     layer.trainable = True

    # Feature extraction
    vgg_gap = GlobalAveragePooling2D()(vgg_base.output)
    resnet_gap = GlobalAveragePooling2D()(resnet_base.output)
    merged = tf.keras.layers.concatenate([vgg_gap, resnet_gap])

    # Enhanced classifier with regularization
    x = Dense(256, activation='relu', kernel_regularizer=l2(0.01))(merged)
    x = BatchNormalization()(x)
    x = Dropout(0.7)(x)
    outputs = Dense(len(CATEGORIES), activation='softmax')(x)

    model = Model(inputs=[vgg_base.input, resnet_base.input], outputs=outputs)
    return model

model = create_model()

# ================== Custom Loss with Class Weights ==================
def weighted_categorical_crossentropy(y_true, y_pred):
    weights = tf.constant(list(class_weight_dict.values()), dtype=tf.float32)
    weighted_loss = tf.reduce_mean(weights * tf.keras.losses.categorical_crossentropy(y_true, y_pred))
    return weighted_loss

# ================== Model Compilation ==================
optimizer = Adam(learning_rate=1e-4)
model.compile(
    optimizer=optimizer,
    loss=weighted_categorical_crossentropy,
    metrics=['accuracy']
)

# ================== Callbacks ==================
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.2, patience=5, verbose=1, min_lr=1e-7
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=10, restore_best_weights=True, verbose=1
    ),
    tf.keras.callbacks.ModelCheckpoint(
        '/content/best_model.h5', save_best_only=True, monitor='val_accuracy'
    )
]

# ================== Training ==================
history = model.fit(
    train_dual_dataset,
    epochs=EPOCHS,
    validation_data=val_dual_dataset,
    callbacks=callbacks,
    steps_per_epoch=len(train_generator),
    validation_steps=len(val_generator),
    class_weight=class_weight_dict
)

# ================== Proper Evaluation ==================
# Get true labels
y_true = val_generator.classes

# Predict with proper length handling
y_pred_probs = model.predict(val_dual_dataset, steps=len(val_generator))
y_pred = np.argmax(y_pred_probs, axis=1)[:len(y_true)]  # Trim to match exact validation size

# Calculate metrics
val_accuracy = accuracy_score(y_true, y_pred)
print(f"\nFinal Validation Accuracy: {val_accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=CATEGORIES))

# ================== Optional: Fine-Tuning ==================
# Uncomment to continue training with unfrozen layers
# for layer in model.layers:
#     if 'vgg16' in layer.name or 'resnet50' in layer.name:
#         layer.trainable = True

# model.compile(
#     optimizer=Adam(learning_rate=1e-6),
#     loss=weighted_categorical_crossentropy,
#     metrics=['accuracy']
# )

# history_fine = model.fit(
#     train_dual_dataset,
#     epochs=5,
#     validation_data=val_dual_dataset,
#     initial_epoch=history.epoch[-1],
#     callbacks=callbacks
# )

# Save final model
model.save('/content/retinal_disease_model_v2.keras')
print("Model saved successfully.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 3384 images belonging to 4 classes.
Found 844 images belonging to 4 classes.
Epoch 1/50


InvalidArgumentError: Graph execution error:

Detected at node compile_loss/weighted_categorical_crossentropy/mul defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run

  File "/usr/local/lib/python3.11/dist-packages/tornado/ioloop.py", line 699, in <lambda>

  File "/usr/local/lib/python